In [12]:
#Importing libraries
#from sklearn.feature_selection.chi2(X, y)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import requests
import seaborn as sns
import pprint, time
from sklearn.model_selection import train_test_split
nltk.download('treebank')
from nltk.tokenize import sent_tokenize
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from collections import Counter
from spellchecker import SpellChecker
import random
from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
import base64
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

nltk.download('punkt')

nltk.download('universal_tagset')

%matplotlib inline

punct = string.punctuation

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\halapets\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\halapets\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\halapets\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


### Bag of Words and LinearSVC (Support Vector Classifier)

In [13]:
#loading spacy
import spacy
nlp = spacy.load("en_core_web_lg")
nlp.max_length = 16000000

In [14]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)
stopwords

['himself',
 'front',
 'full',
 'her',
 'get',
 'ten',
 'twelve',
 'was',
 'done',
 'an',
 'together',
 'beside',
 'might',
 'even',
 'since',
 'within',
 '’re',
 'who',
 'amount',
 'nothing',
 'thereby',
 'next',
 'those',
 'using',
 'third',
 'however',
 'through',
 'about',
 'may',
 'under',
 'around',
 'until',
 'if',
 'due',
 'fifteen',
 'would',
 'no',
 'whence',
 'whose',
 'below',
 'go',
 'from',
 'did',
 'against',
 '‘m',
 'almost',
 'been',
 'beyond',
 'their',
 'therein',
 'first',
 'move',
 'any',
 'the',
 'at',
 'same',
 'latter',
 '’s',
 'hereby',
 'further',
 '‘ve',
 'side',
 'your',
 'also',
 'down',
 'each',
 '‘d',
 'all',
 'anyway',
 'some',
 'ever',
 'wherein',
 'have',
 'yours',
 'whereafter',
 'whither',
 'twenty',
 'eleven',
 'well',
 'alone',
 'rather',
 'and',
 'fifty',
 'myself',
 'therefore',
 'thus',
 'always',
 'namely',
 'whenever',
 'herein',
 'a',
 'bottom',
 'anywhere',
 'indeed',
 'whether',
 'others',
 'during',
 'part',
 "'d",
 'off',
 'ourselves',
 '

In [15]:
#reading data
LatestMobileReviews1 = pd.read_csv('C:\\Users\\halapets\\Documents\\LJMU Research_Sentiment Analysis\\LatestMobileReviews.csv',encoding='latin1')
LatestMobileReviews1.head()

,Product,ReviewUpdated,Rating,Date,Website,TitleGoodBad,ProductPrice,ProductPriceRange,Year,positive_negative,...,Positive_Negative_Tag,LengthofReviews,SentimentPolarity,SentimentSubjectivity,SentimentPolarityClass,AvgSentPolarity,AvgSentimentPolarityClass,AvgSentimentSubjectivity,ReviewUpdated1,ReviewUpdated2
0,Xiaomi Redmi Note 3,"['yet', 'another', 'value', 'money', 'product'...",3.0,"Jul 17, 2016 10:59 PM",MouthShut,Yet another Value for Money product from Xiaomi,9999,Budget Range,2016.0,1,...,Positive Rating,344,-0.028571,0.576190,Negative Rating,0.403533,Positive Rating,NaN,Yet another Value for Money product from Xiaom...,Yet another Value for Money product from Xiaom...
1,Xiaomi Redmi Note 3,"['awesome', 'marvelous', 'mobile', 'go', 'valu...",5.0,"Jul 17, 2016 10:42 PM",MouthShut,Awesome,9999,Budget Range,2016.0,1,...,Positive Rating,276,0.780000,0.640000,Positive Rating,0.403533,Positive Rating,NaN,"Awesome Marvelous Mobile just go for it., Val...","Awesome Marvelous Mobile just go for it., Val..."
2,Xiaomi Redmi Note 3,"['power', 'pack', 'review', 'redmi', 'note', '...",5.0,"Jul 17, 2016 06:18 PM",MouthShut,Power pack review on Redmi Note 3,9999,Budget Range,2016.0,1,...,Positive Rating,322,0.474444,0.477037,Positive Rating,0.403533,Positive Rating,NaN,Power pack review on Redmi Note 3 REDMI NOTE ...,Power pack review on Redmi Note 3 REDMI NOTE ...
3,Xiaomi Redmi Note 3,"['flagship', 'smartfone', 'phone', 'looks', 'v...",4.0,"Jul 17, 2016 04:44 PM",MouthShut,Flagship smartfone under 13000rs,9999,Budget Range,2016.0,1,...,Positive Rating,333,0.502500,0.832500,Positive Rating,0.403533,Positive Rating,NaN,Flagship smartfone under 13000rs This phone l...,Flagship smartfone under 13000rs This phone l...
4,Xiaomi Redmi Note 3,"['boon', 'mobile', 'master', 'piece', 'economi...",5.0,"Jul 17, 2016 12:32 PM",MouthShut,Boon for mobile,9999,Budget Range,2016.0,1,...,Positive Rating,316,0.448571,0.627857,Positive Rating,0.403533,Positive Rating,NaN,Boon for mobile Master piece in such a econom...,Boon for mobile Master piece in such a econom...


In [36]:
#tokenization and text preprocessing
def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-" or token.pos_ != "AUX" or token.pos_ != "NUM" or token.pos_ != "SYM" or token.pos_ != "ADP"  :
            temp = token.lemma_.lower().strip()
        tokens.append(temp)
        
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct and token.isalpha():
            cleaned_tokens.append(token)
    return cleaned_tokens

In [37]:
from sklearn.svm import LinearSVC#change

#from sklearn.model_selection import GridSearchCV 

In [38]:
#word vectorization using BOW
BOW = CountVectorizer(tokenizer = text_data_cleaning)
#create base classifier
classifier = LinearSVC()

In [39]:
X = LatestMobileReviews1[['Product','ReviewUpdated1']]
y= LatestMobileReviews1 ['positive_negative']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1001)

In [41]:
X_train.columns

Index(['Product', 'ReviewUpdated1'], dtype='object')

In [42]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((112039, 2), (112039,), (48018, 2), (48018,))

In [43]:
#creating a pipeline of steps to be followed for classifier
clf_BOW_SVM = Pipeline([('BOW', BOW), ('clf', classifier)])

In [44]:
#fitting model
clf_BOW_SVM.fit(X_train['ReviewUpdated1'], y_train)


C:\Users\halapets\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('BOW',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_data_cleaning at 0x000001EA8FCDA168>,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_cla

In [45]:
#Evaluating model on test data
y_pred_BOW_SVM = clf_BOW_SVM.predict(X_test['ReviewUpdated1'])
X_test.columns

Index(['Product', 'ReviewUpdated1'], dtype='object')

In [46]:
target_names = ['class 0', 'class 1']
ClassificationReport_BOW_SVM=classification_report(y_test, y_pred_BOW_SVM, target_names=target_names)
ClassificationReport_BOW_SVM

#'              precision    recall  f1-score   support\n\n     
#class 0       0.70      0.58      0.63      7791\n     
#class 1       0.92      0.95      0.94     40227\n\n    
#accuracy                          0.89     48018\n   
#macro avg     0.81      0.76      0.78     48018\n
#weighted avg  0.88      0.89      0.89     48018\n'

'              precision    recall  f1-score   support\n\n     class 0       0.70      0.58      0.63      7791\n     class 1       0.92      0.95      0.94     40227\n\n    accuracy                           0.89     48018\n   macro avg       0.81      0.76      0.78     48018\nweighted avg       0.88      0.89      0.89     48018\n'

In [47]:
ConfusionMatrix_BOW_SVM=confusion_matrix(y_test, y_pred_BOW_SVM)
ConfusionMatrix_BOW_SVM

array([[ 4497,  3294],
       [ 1952, 38275]], dtype=int64)

In [48]:
accuracy_score(y_test, y_pred_BOW_SVM) #0.890749302344954 accuracy of base model

0.890749302344954

In [49]:
y_pred_BOW_SVM = pd.DataFrame(y_pred_BOW_SVM,X_test['Product'])
y_pred_BOW_SVM.rename(columns={"BOW_SVM":"Tags_BOW_SVM"}, inplace = True)
y_pred_BOW_SVM.head(5)
len(y_pred_BOW_SVM)#48019
y_pred_BOW_SVM.to_csv("y_pred_BOW_SVM.csv", header= True)

In [50]:
# defining parameter range 
#param_grid = {'C': [0.1, 1, 10, 100, 1000]} 

In [85]:
# Instantiate the grid search model
#grid_search_BOW_SVM = GridSearchCV(classifier, param_grid,cv = 3, verbose = 1,n_jobs=1)

In [87]:
# Fit the grid search to the data
#grid_search_BOW_SVM.fit(BOW.transform(X_train), y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\halapets\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  4.0min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=1,
             param_grid={'C': [0.1, 1, 10, 100, 1000]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=1)

In [92]:
#best_grid = grid_search_BOW_SVM.best_estimator_
#best_grid

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [96]:
#y_pred_grid_search_BOW_SVM = grid_search_BOW_SVM.predict(BOW.transform(X_test))

In [103]:
#target_names = ['class 0', 'class 1']
#classification_report_BOW_SVM_gridsearch=classification_report(y_test, y_pred_grid_search_BOW_SVM, target_names=target_names)
#classification_report_BOW_SVM_gridsearch

'              precision    recall  f1-score   support\n\n     class 0       0.80      0.65      0.72     16505\n     class 1       0.91      0.96      0.93     61332\n\n    accuracy                           0.89     77837\n   macro avg       0.85      0.80      0.82     77837\nweighted avg       0.89      0.89      0.89     77837\n'

In [104]:
#ConfusionMatrix_BOW_SVM_gridsearch=confusion_matrix(y_test, y_pred_grid_search_BOW_SVM)
#ConfusionMatrix_BOW_SVM_gridsearch

array([[10708,  5797],
       [ 2737, 58595]], dtype=int64)

In [105]:
#accuracy_score(y_test, y_pred_grid_search_BOW_SVM) #0.8903606254095097 accuracy of base model

0.8903606254095097

### TF-IDF Vectorization and Logistic Regression Classifier


In [51]:
from sklearn import linear_model
#word vectorization using BOW
BOW = CountVectorizer(tokenizer = text_data_cleaning)
#create base classifier
LR = linear_model.LogisticRegression()

#creating a pipeline of steps to be followed for classifier
clf_BOW_LR = Pipeline([('BOW', BOW), ('clf', LR)])#change

#fitting model
clf_BOW_LR.fit(X_train['ReviewUpdated1'], y_train)



C:\Users\halapets\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('BOW',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_data_cleaning at 0x000001EA8FCDA168>,
                                 vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                

In [52]:
#Evaluating model on test data
y_pred_BOW_LR = clf_BOW_LR.predict(X_test['ReviewUpdated1'])


In [53]:
target_names = ['class 0', 'class 1']
ClassificationReport_BOW_LR=classification_report(y_test, y_pred_BOW_LR, target_names=target_names)
ClassificationReport_BOW_LR

#'              precision    recall  f1-score   support\n\n     
#class 0       0.76      0.58      0.66      7791\n     
#class 1       0.92      0.97      0.94     40227\n\n    
#accuracy                          0.90     48018\n   
#macro avg     0.84      0.77      0.80     48018\n
#weighted avg  0.90      0.90      0.90     48018\n'

'              precision    recall  f1-score   support\n\n     class 0       0.76      0.58      0.66      7791\n     class 1       0.92      0.97      0.94     40227\n\n    accuracy                           0.90     48018\n   macro avg       0.84      0.77      0.80     48018\nweighted avg       0.90      0.90      0.90     48018\n'

In [54]:
ConfusionMatrix_BOW_LR=confusion_matrix(y_test, y_pred_BOW_LR)
ConfusionMatrix_BOW_LR

array([[ 4520,  3271],
       [ 1395, 38832]], dtype=int64)

In [55]:
accuracy_score(y_test, y_pred_BOW_LR) #0.9028281061268691 accuracy of base model

0.9028281061268691

In [56]:
y_pred_BOW_LR = pd.DataFrame(y_pred_BOW_LR,X_test['Product'])
y_pred_BOW_LR.rename(columns={0:"Tags_BOW_LR"}, inplace = True)
y_pred_BOW_LR.head(5)
#len(y_pred_BOW_LR)
y_pred_BOW_LR.to_csv("y_pred_BOW_LR.csv",header = True)



In [57]:
#Create Hyperparameter Search Space 
# Create regularization penalty space
#penalty = ['l1', 'l2']

# Create regularization hyperparameter space
#C = np.logspace(0, 4, 10)

# Create hyperparameter options
#hyperparameters = dict(C=C, penalty=penalty)

In [ ]:
# Create grid search using 5-fold cross validation
#grid_search_BOW_LR = GridSearchCV(classifier, hyperparameters, cv=5, verbose=0)

In [ ]:
# Fit the grid search to the data
#grid_search_BOW_LR.fit(BOW.transform(X_train), y_train)

In [ ]:

#best_grid_BOW_LR = grid_search_BOW_LR.best_estimator_

#best_grid_BOW_LR

In [ ]:
#y_pred_grid_search_BOW_LR = grid_search_BOW_LR.predict(BOW.transform(X_test))

In [ ]:
#target_names = ['class 0', 'class 1']
#ClassificationReport_BOW_LR_gridsearch=classification_report(y_test, y_pred_grid_search_BOW_LR, target_names=target_names)

In [ ]:
#ConfusionMatrix_BOW_LR_gridsearch=confusion_matrix(y_test, y_pred_grid_search_BOW_LR)

In [74]:
#accuracy_score(y_test, y_pred_grid_search_BOW_LR) #0.8903606254095097 accuracy of base model


102549    Samsung j2 I have a Samsung j2 2016 mobile I b...
115553    All about OPPO A37... This phone is one of the...
23301     The best budget SMART PHONE Xiaomi redmi note ...
158021    Best Budget smartphone in the range of Rs. 699...
112653    Perfect mobile at 7K It is a prefect mobile wh...
Name: ReviewUpdated1, dtype: object

### TF-IDF Vectorization and Naive Bayes Classifier


In [58]:
from sklearn.naive_bayes import BernoulliNB
#word vectorization using BOW
BOW = CountVectorizer(tokenizer = text_data_cleaning)
#create base classifier
NB = BernoulliNB()

#creating a pipeline of steps to be followed for classifier
clf_BOW_NB = Pipeline([('BOW', BOW), ('clf', NB)])#change

#fitting model
clf_BOW_NB.fit(X_train['ReviewUpdated1'], y_train)

Pipeline(memory=None,
         steps=[('BOW',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_data_cleaning at 0x000001EA8FCDA168>,
                                 vocabulary=None)),
                ('clf',
                 BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None,
                             fit_prior=True))],
         verbose=False)

In [59]:
#Evaluating model on test data
y_pred_BOW_NB = clf_BOW_NB.predict(X_test['ReviewUpdated1'])

In [60]:
target_names = ['class 0', 'class 1']
ClassificationReport_BOW_NB=classification_report(y_test, y_pred_BOW_NB, target_names=target_names)
ClassificationReport_BOW_NB

#'              precision    recall  f1-score   support\n\n     
#class 0       0.72      0.50      0.59      7791\n     
#class 1       0.91      0.96      0.93     40227\n\n    
#accuracy                          0.89     48018\n   
#macro avg     0.82      0.73      0.76     48018\n
#weighted avg  0.88      0.89      0.88     48018\n'

'              precision    recall  f1-score   support\n\n     class 0       0.72      0.50      0.59      7791\n     class 1       0.91      0.96      0.93     40227\n\n    accuracy                           0.89     48018\n   macro avg       0.82      0.73      0.76     48018\nweighted avg       0.88      0.89      0.88     48018\n'

In [61]:
ConfusionMatrix_BOW_NB=confusion_matrix(y_test, y_pred_BOW_NB)
ConfusionMatrix_BOW_NB

array([[ 3868,  3923],
       [ 1487, 38740]], dtype=int64)

In [62]:
accuracy_score(y_test, y_pred_BOW_NB) #0.8873339164479986 accuracy of base model

0.8873339164479986

In [63]:
y_pred_BOW_NB = pd.DataFrame(y_pred_BOW_NB,X_test['Product'])
y_pred_BOW_NB.rename(columns={0:"Tags_BOW_NB"}, inplace = True)
y_pred_BOW_NB.head(5)
#len(y_pred_BOW_NB)
y_pred_BOW_NB.to_csv("y_pred_BOW_NB.csv",header = True)

### TF-IDF Vectorization and Decision Tree Classifier

In [64]:
from sklearn.tree import DecisionTreeClassifier 
#from sklearn.datasets import make_classification
#word vectorization using BOW
BOW = CountVectorizer(tokenizer = text_data_cleaning)
#create base classifier
cart = DecisionTreeClassifier(criterion = "gini", 
            random_state = 100) 

#creating a pipeline of steps to be followed for classifier
clf_BOW_CART = Pipeline([('BOW', BOW), ('clf', cart)])#change

#fitting model
clf_BOW_CART.fit(X_train['ReviewUpdated1'], y_train)



#X, y = make_classification(n_samples=1000, n_features=4,
#                            n_informative=2, n_redundant=0,
#                            random_state=0, shuffle=False)






Pipeline(memory=None,
         steps=[('BOW',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_data_c...A8FCDA168>,
                                 vocabulary=None)),
                ('clf',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_leaf_nodes=None,
                     

In [65]:
#Evaluating model on test data
y_pred_BOW_CART = clf_BOW_CART.predict(X_test['ReviewUpdated1'])
y_pred_BOW_CART

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [66]:
target_names = ['class 0', 'class 1']
ClassificationReport_BOW_CART=classification_report(y_test, y_pred_BOW_CART, target_names=target_names)
ClassificationReport_BOW_CART

#'              precision    recall  f1-score   support\n\n     
#class 0       0.57      0.49      0.53      7791\n     
#class 1       0.90      0.93      0.92     40227\n\n    
#accuracy                          0.86     48018\n   
#macro avg     0.74      0.71      0.72     48018\n
#weighted avg  0.85      0.86      0.85     48018\n'

'              precision    recall  f1-score   support\n\n     class 0       0.57      0.49      0.53      7791\n     class 1       0.90      0.93      0.92     40227\n\n    accuracy                           0.86     48018\n   macro avg       0.74      0.71      0.72     48018\nweighted avg       0.85      0.86      0.85     48018\n'

In [67]:
ConfusionMatrix_BOW_CART=confusion_matrix(y_test, y_pred_BOW_CART)
ConfusionMatrix_BOW_CART

array([[ 3817,  3974],
       [ 2864, 37363]], dtype=int64)

In [68]:
accuracy_score(y_test, y_pred_BOW_CART) #0.8575950685159732 accuracy of base model

0.8575950685159732

In [69]:
y_pred_BOW_CART = pd.DataFrame(y_pred_BOW_CART,X_test['Product'])
y_pred_BOW_CART.rename(columns={0:"Tags_BOW_CART"}, inplace = True)
y_pred_BOW_CART.head(5)
#len(y_pred_BOW_CART)
y_pred_BOW_CART.to_csv("y_pred_BOW_CART.csv",header = True)

In [98]:
#random_grid = {'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
#              'max_features': [200,300]}

In [ ]:
# Create grid search using 5-fold cross validation
#rf_random  = RandomizedSearchCV(rf, random_grid,n_iter = 100, cv=5, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
# Fit the grid search to the data
#rf_random.fit(BOW.transform(X_train), y_train)

In [ ]:

#best_grid_BOW_RF = rf_random.best_params_

#best_grid_BOW_RF

In [ ]:
#y_pred_grid_search_BOW_RF = rf_random.predict(BOW.transform(X_test))

In [ ]:
#target_names = ['class 0', 'class 1']
#ClassificationReport_BOW_RF_gridsearch=classification_report(y_test, y_pred_grid_search_BOW_RF, target_names=target_names)

In [ ]:
#ConfusionMatrix_BOW_RF_gridsearch=confusion_matrix(y_test, y_pred_grid_search_BOW_RF)

In [ ]:
#accuracy_score(y_test, y_pred_grid_search_BOW_RF) #0.8903606254095097 accuracy of base model

In [ ]:
#Ensemble Model based on Individual Classifiers using Majority of voting

In [70]:
y_pred_BOW_NB = pd.read_csv("y_pred_BOW_NB.csv",index_col=None)
y_pred_BOW_LR= pd.read_csv("y_pred_BOW_LR.csv",index_col=None)
y_pred_BOW_CART= pd.read_csv("y_pred_BOW_CART.csv",index_col=None)
y_pred_BOW_SVM= pd.read_csv("y_pred_BOW_SVM.csv",index_col=None)

In [71]:
y_pred_BOW_ensb=y_pred_BOW_NB.merge(y_pred_BOW_LR, how = 'inner',left_index = True, right_index = True, on = 'Product')


In [72]:
y_pred_BOW_ensb = y_pred_BOW_ensb.merge(y_pred_BOW_CART, how='inner', left_index = True, right_index = True, on = 'Product')

In [73]:
y_pred_BOW_ensb.shape

(48018, 4)

In [74]:

y_pred_BOW_ensb.head()

,Product,Tags_BOW_NB,Tags_BOW_LR,Tags_BOW_CART
0,Xiaomi Redmi 4A,1,1,1
1,Lenovo K3 Note,1,1,0
2,Samsung Galaxy Note 3,1,1,1
3,Xiaomi Redmi Note 3,1,1,1
4,Xiaomi Redmi 3S Prime,1,1,1


In [75]:
y_pred_BOW_ensb = y_pred_BOW_ensb.merge(y_pred_BOW_SVM, how='inner', left_index = True, right_index = True, on = 'Product')

In [76]:

y_pred_BOW_ensb.columns

Index(['Product', 'Tags_BOW_NB', 'Tags_BOW_LR', 'Tags_BOW_CART',
       'Tags_BOW_SVM'],
      dtype='object')

In [77]:
#Adding weights on classifiers based on the performance of the classifiers in terms of accuracy, precision and recall
#Naive Bayes and Decision Tree has been added a weight of 0.5 each
y_pred_BOW_ensb["Total Votes"] = (0.5*y_pred_BOW_ensb["Tags_BOW_NB"])+y_pred_BOW_ensb["Tags_BOW_LR"] + (0.5*y_pred_BOW_ensb["Tags_BOW_CART"])+y_pred_BOW_ensb["Tags_BOW_SVM"]


In [78]:
y_pred_BOW_ensb["Majority of Votes"]  = [1 if x >=2 else 0 for x in y_pred_BOW_ensb["Total Votes"]] 

In [79]:
target_names = ['class 0', 'class 1']
ClassificationReport_BOW_Ensemble=classification_report(y_test, y_pred_BOW_ensb["Majority of Votes"], target_names=target_names)
ClassificationReport_BOW_Ensemble

#'              precision    recall  f1-score   support\n\n     
#class 0       0.75      0.61      0.67      7791\n     
#class 1       0.93      0.96      0.94     40227\n\n    
#accuracy                          0.90     48018\n   
#macro avg     0.84      0.78      0.81     48018\n
#weighted avg  0.90      0.90      0.90     48018\n'



'              precision    recall  f1-score   support\n\n     class 0       0.75      0.61      0.67      7791\n     class 1       0.93      0.96      0.94     40227\n\n    accuracy                           0.90     48018\n   macro avg       0.84      0.78      0.81     48018\nweighted avg       0.90      0.90      0.90     48018\n'

In [80]:
ConfusionMatrix_BOW_Ensemble=confusion_matrix(y_test, y_pred_BOW_ensb["Majority of Votes"])
ConfusionMatrix_BOW_Ensemble

array([[ 4715,  3076],
       [ 1586, 38641]], dtype=int64)

In [81]:
accuracy_score(y_test, y_pred_BOW_ensb["Majority of Votes"]) #0.9029114082219167 accuracy of base model

0.9029114082219167